# Web Scraping with Beautiful Soup

This tutorial will provide a very basic introduction to using the Beautiful Soup package to scrape text data from the web. 

## Installation

In a terminal, install Beautiful Soup if necessary by running <pre><code>conda install beautiful-soup</code></pre>

## Retrieving HTML

The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML underlying any website is not interesting to us.

We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

### urllib2
urllib2 is a module for working with urls, we will use it to open connections to urls and retrieve the webpage source.

In [148]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false').read()
soup = BeautifulSoup(page)
print type(soup)

<class 'bs4.BeautifulSoup'>


The soup object contains all of the HTML in the original document.

In [149]:
soup.prettify()[0:1000]

u'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml">\n <head>\n  <script src="http://sports-ak.espn.go.com/sports/optimizely.js">\n  </script>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any"/>\n  <meta content="#CC0000" name="theme-color"/>\n  <script type="text/javascript">\n   if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {\n        window.location = \'http://a.m.espn.go.com/mobilecache/general/apps/sc\';\n    }\n  </script>\n  <title>\n   2015-16 Regular Season NBA Player Stats and League Leaders - Scoring Per Game - National Basketball Association - ESPN\n  </title>\n  <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>\n  <meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>\n  <meta content="noodp" name="googlebot"/>\n  <meta co

## Parsing HTML

By "parsing HTML", we mean pulling out only the relevant tags/attributes for our analysis.  What Beautiful Soup does is provide a handy bunch of methods for doing this efficiently.

### find method

The find method will search for and return the first tag matching your corresponding search criteria, if it exists.  You can specify tag and attribute info etc.  There is also a findAll method that will return a collection of tags matching your query.

In [150]:
table_div = soup.find(id='my-players-table')
print table_div.prettify()[0:4000]

<div class="col-main" id="my-players-table">
 <div class="mod-container mod-table">
  <div class="mod-header stathead">
   <h4>
    Points Per Game Leaders - All Players
   </h4>
  </div>
  <div class="mod-content">
   <table cellpadding="3" cellspacing="1" class="tablehead">
    <tr align="right" class="colhead">
     <td align="left" style="width:20px;">
      RK
     </td>
     <td align="left">
      PLAYER
     </td>
     <td align="left">
      TEAM
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
       GP
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
       MPG
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
       PTS

Notice that the object returned by the find method is just another inner HTML structure, which we can step through just like we would have with the original soup object.  We've gone to the location in the webpage where the table that we seek starts, now we can use find again to get to the table data.

In [151]:
table = table_div.find("table")
print table

<table cellpadding="3" cellspacing="1" class="tablehead">
<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/play

### Searching by Attributes

Now that we have the table object, we need to step through the rows.  First we'll find the header row so we can populate what the field names will be in our data.  Here we're searching for tags under the table tag whose class attritbute is "colhead".  

In [152]:
table_head = table.find(attrs={"class":'colhead'})
print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
   PTS
  </a>
 </td>
 <td>
  <span title="Field Goals Made-Attempted Per Game">
   FGM-FGA
  </span>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
   FG%
  </a>
 </td>
 <td>
  <span title="3-Point Field Goals Made-Attempted Per Game">
   3PM-3PA
  </span>
 </td>
 <t

Now we find the actual values by searching for the 'td' tags, which is the tag for table data.

In [153]:
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td>, <td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td>, <td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qualified/false" t

Finally, we step through these columns and save them to a list to be used later.  We'll ignore the rank column (RK) because that doesn't give us anything we want later.  We also separate the **PLAYER** column into **PLAYER** and **POSITION**.

In [154]:
cols = []
for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


### Stepping Through a Table

The table rows are indicated by the tag 'tr'.  Again we can find them all and iterate through them.  Within each row we iterate through the respective columns.

In [155]:
table_rows = table.findAll('tr')
print table_rows

[<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qu

We will save our results in 2 different ways to demonstrate how we can handle both.  The first will be a list of dicts where the key is the field name and the value is the field value.  The second will just be a list of lists of stats with no field name values (we've already defined them earlier).

In [156]:
players_stats_dicts = []
players_stats_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    players_stats_dicts.append(player_stats)
print players_stats_dicts[0:5] 
print players_stats_array[0:5]

[{u'FGM-FGA': u'10.6-19.9', u'MPG': u'33.8', u'FTM-FTA': u'6.0-6.6', u'FG%': u'.535', u'GP': u'8', u'PLAYER': u'Stephen Curry', u'FT%': u'.906', u'TEAM': u'GS', u'3PM-3PA': u'5.1-10.9', 'POSITION': u'PG', u'PTS': u'32.4', u'3P%': u'.471'}, {u'FGM-FGA': u'8.1-21.6', u'MPG': u'39.4', u'FTM-FTA': u'10.9-12.7', u'FG%': u'.377', u'GP': u'7', u'PLAYER': u'James Harden', u'FT%': u'.854', u'TEAM': u'HOU', u'3PM-3PA': u'2.6-10.7', 'POSITION': u'SG', u'PTS': u'29.7', u'3P%': u'.240'}, {u'FGM-FGA': u'11.3-19.4', u'MPG': u'35.6', u'FTM-FTA': u'6.0-7.4', u'FG%': u'.581', u'GP': u'7', u'PLAYER': u'Blake Griffin', u'FT%': u'.808', u'TEAM': u'LAC', u'3PM-3PA': u'0.0-0.4', 'POSITION': u'PF', u'PTS': u'28.6', u'3P%': u'.000'}, {u'FGM-FGA': u'9.6-19.1', u'MPG': u'34.8', u'FTM-FTA': u'6.3-7.0', u'FG%': u'.503', u'GP': u'8', u'PLAYER': u'Kevin Durant', u'FT%': u'.893', u'TEAM': u'OKC', u'3PM-3PA': u'2.6-5.9', 'POSITION': u'SF', u'PTS': u'28.1', u'3P%': u'.447'}, {u'FGM-FGA': u'9.6-20.6', u'MPG': u'36.6', u

Here we've used the zip function to combine pairs of lists into tuples, and then transformed that into a dict to get a dictionary of FIELD --> VALUE for every player in the table.

Beautiful Soup has many other features, including the ability to step up, down, and sideways in the HTML tree and basically search for any tags, attributes, or values.  For more, take a look at the [Documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## Load into Pandas

Let's load our scraped data into Pandas and take a look at it.  Here is the first way we can do it, simply directly from the dictionary we defined.

In [157]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.471,5.1-10.9,.535,10.6-19.9,.906,6.0-6.6,8,33.8,Stephen Curry,PG,32.4,GS
1,.240,2.6-10.7,.377,8.1-21.6,.854,10.9-12.7,7,39.4,James Harden,SG,29.7,HOU
2,.000,0.0-0.4,.581,11.3-19.4,.808,6.0-7.4,7,35.6,Blake Griffin,PF,28.6,LAC
3,.447,2.6-5.9,.503,9.6-19.1,.893,6.3-7.0,8,34.8,Kevin Durant,SF,28.1,OKC
4,.429,3.8-8.8,.467,9.6-20.6,.900,4.5-5.0,8,36.6,Damian Lillard,PG,27.5,POR


Now here is a 2nd way we can do it.  We convert the 2D stats array into a numpy array and create a Pandas dataframe from it along with the list of column headers we defined earlier.

In [158]:
np_array = np.array(players_stats_array)
df = pd.DataFrame(np_array, columns=cols)
df.head()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Stephen Curry,PG,GS,8,33.8,32.4,10.6-19.9,.535,5.1-10.9,.471,6.0-6.6,.906
1,James Harden,SG,HOU,7,39.4,29.7,8.1-21.6,.377,2.6-10.7,.240,10.9-12.7,.854
2,Blake Griffin,PF,LAC,7,35.6,28.6,11.3-19.4,.581,0.0-0.4,.000,6.0-7.4,.808
3,Kevin Durant,SF,OKC,8,34.8,28.1,9.6-19.1,.503,2.6-5.9,.447,6.3-7.0,.893
4,Damian Lillard,PG,POR,8,36.6,27.5,9.6-20.6,.467,3.8-8.8,.429,4.5-5.0,.900


## Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

The general steps should be as follows:
- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)
- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.
- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).
- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result

In [159]:
from bs4 import BeautifulSoup as BS
import urllib2
page_url = "http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false"
page = urllib2.urlopen(page_url).read()
bs_page = BS(page)
page_numbers = bs_page.find(attrs={"class":"page-numbers"})
players_shooting_stats_array = []
players_assists_stats_array = []
cols = []
table_id = "my-players-table"
table_div = bs_page.find(id=table_id)
table = table_div.find("table")


def get_cols():
    table_id = "my-players-table"
    table_div = bs_page.find(id=table_id)
    table = table_div.find("table")
    table_head = table.find(attrs={"class":'colhead'})
    header_cols = table_head.findAll('td')
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')

def get_data(stats_array):
    table_id = "my-players-table"
    table_div = bs_page.find(id=table_id)
    table = table_div.find("table")
    table_rows = table.findAll('tr')
    for row in table_rows:
        if row.attrs['class'][0]=='colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        stats_array.append(player_stats)
get_cols()
get_data(players_shooting_stats_array)

In [160]:
for x in range(1,11):
    #if page_numbers.string.split(" ")[0] != page_numbers.string.split(" ")[2]: #test to see if we are on last page
        y = 1
        y += x * 40
        iter_page_url = page_url + "/count/" + str(y)
        page = urllib2.urlopen(iter_page_url).read()
        bs_page = BS(page)
        page_numbers = bs_page.find(attrs={"class":"page-numbers"})
        get_data(players_shooting_stats_array)
np_array = np.array(players_shooting_stats_array)
df_player_stats = pd.DataFrame(np_array, columns=cols)
df_player_stats.loc[400:408]


,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
400,Reggie Bullock,SF,DET,4,6.3,0.0,0.0-1.0,.000,0.0-0.8,.000,0.0-0.0,.000
401,James Ennis,SF,MIA,3,2.3,0.0,0.0-0.0,.000,0.0-0.0,.000,0.0-0.0,.000
402,Jarnell Stokes,PF,MEM,2,2.0,0.0,0.0-0.0,.000,0.0-0.0,.000,0.0-0.0,.000
403,Salah Mejri,C,DAL,4,5.0,0.0,0.0-0.3,.000,0.0-0.0,.000,0.0-0.0,.000
404,Sam Dekker,SF,HOU,3,2.0,0.0,0.0-0.0,.000,0.0-0.0,.000,0.0-0.0,.000
405,Delon Wright,PG,TOR,2,2.5,0.0,0.0-1.0,.000,0.0-0.5,.000,0.0-0.0,.000
406,James Young,SG,BOS,1,1.0,0.0,0.0-0.0,.000,0.0-0.0,.000,0.0-0.0,.000
407,Cristiano Felicio,PF,CHI,1,1.0,0.0,0.0-0.0,.000,0.0-0.0,.000,0.0-0.0,.000


In [164]:
assists_url = "http://espn.go.com/nba/statistics/player/_/stat/assists/qualified/false"
players_assists_stats_array =[]
cols = []
page = urllib2.urlopen(assists_url).read()
bs_page = BS(page)
get_cols()
y = 1
for x in range(0,11):   
        y = x * 40
        iter_page_url = assists_url + "/count/" + str(y)
        page = urllib2.urlopen(iter_page_url).read()
        bs_page = BS(page)
        page_numbers = bs_page.find(attrs={"class":"page-numbers"})
        get_data(players_assists_stats_array)

np_array2 = np.array(players_assists_stats_array)
df_player_assists = pd.DataFrame(np_array2, columns=cols)
df_player_assists.loc[400:407]
       


,PLAYER,POSITION,TEAM,GP,MPG,AST,APG,TO,TOPG,AP48M,AST/TO
400,Sam Dekker,SF,HOU,3,2.0,0,0.0,0,0.0,0.0,0.00
401,Anthony Bennett,PF,TOR,5,7.0,0,0.0,3,0.6,0.0,0.00
402,Lucas Nogueira,C,TOR,1,3.0,0,0.0,0,0.0,0.0,0.00
403,Walter Tavares,C,ATL,2,1.0,0,0.0,0,0.0,0.0,0.00
404,Delon Wright,PG,TOR,2,2.5,0,0.0,1,0.5,0.0,0.00
405,James Young,SG,BOS,1,1.0,0,0.0,0,0.0,0.0,0.00
406,Rashad Vaughn,SG,MIL,5,8.2,0,0.0,2,0.4,0.0,0.00
407,Luis Montero,SG,POR,2,3.0,0,0.0,1,0.5,0.0,0.00


In [170]:
df_player_stats.head()
df_player_stats.join(df_player_assists,on="PLAYER",lsuffix="s",rsuffix="a")
df_player_stats.head()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Stephen Curry,PG,GS,8,33.8,32.4,10.6-19.9,.535,5.1-10.9,.471,6.0-6.6,.906
1,James Harden,SG,HOU,7,39.4,29.7,8.1-21.6,.377,2.6-10.7,.240,10.9-12.7,.854
2,Blake Griffin,PF,LAC,7,35.6,28.6,11.3-19.4,.581,0.0-0.4,.000,6.0-7.4,.808
3,Kevin Durant,SF,OKC,8,34.8,28.1,9.6-19.1,.503,2.6-5.9,.447,6.3-7.0,.893
4,Damian Lillard,PG,POR,8,36.6,27.5,9.6-20.6,.467,3.8-8.8,.429,4.5-5.0,.900


In [172]:
df_player_stats.describe()



,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
count,408,408,408,408,408,408,408,408,408,408,408,408
unique,408,5,30,9,223,164,353,201,208,106,232,114
top,Paul Pierce,PF,MIA,7,7.0,0.0,0.0-0.0,.000,0.0-0.0,.000,0.0-0.0,.000
freq,1,98,15,135,7,24,13,29,100,143,69,74
